<a href="https://colab.research.google.com/github/flying-bear/Style-Transfer/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Style Trasfer

## Plan

* Problem setting
* Dataset
* Models
  * Autoencoder
  * AdaIN
  * ArtGAN (simplified)
  * ArtGAN
* Problems
  * Evaluation Protocols
  * Batch vs Instance Normalization
  * Local to Global Statistic Normalization
* Additional Ideas
  * Local Context Normalization
  * Noise Layers
  * Attention Experiment
* References

All code for this project is available at [GitHub](https://https://github.com/flying-bear/Style-Transfer).



# Problem Setting
**Style transfer** is a process that renders a content image in the style of another image. 
There are two general settings of the problem:
* *pre-trained style transfer*, where the model is trained to transfer pre-set style onto any content image
* *general style transfer*, where two images, of style and of content, and the style is rendered onto the content

# Dataset and image preprocessing
The **style** dataset consisted of 113 RGB images by Owen Gent downloaded from the authors [behance](https://www.behance.net/owengent).

The **content** dataset consisted of 800 RGB images from the [DIV2K dataset](https://data.vision.ee.ethz.ch/cvl/DIV2K/) of diverse 2K resolution images collected by restricted image crawl (see [Agustsson & Timofte (2017)](https://people.ee.ethz.ch/~timofter/publications/Agustsson-CVPRW-2017.pdf)).

Each image was first compressed to width of 512p and then randomly corpped to the size of 224 $\times$ 224 p. This preprocessing allows to use each image multiple times and also allow to deal with smaller resolution, which decreases the network complexity & training time.


# Autoencoder
Initially, we train a simple autoencoder, using the first 30 layers of VGG  (pre-trained, frozen) as the encoder to check if it possible to train such a thing in Colab on GPU. The decoder is a reversed vgg architechture, upsampling the encoded image.

* **pixelwise loss**: `MSE[x - D(E(x))]`, forcing the generated image to pixel-wise coincide with the input, learns very fast, *10 minutes* already produces fine results. No pre-trained model was used for thsi loss.
* **content loss**: `MSE[E(x) - E(D(E(X))]`, forcing the generated image features to coincide with the input features, still, surprisingly, learns fast. VGG layesr were used for the encoder part of the network in the content-loss setting.

The implementation can be found on [GitHub](https://github.com/flying-bear/Style-Transfer/blob/main/AE.ipynb).

# AdaIN

<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/AdaIN_scheme.png?raw=true">


[AdaIN](https://arxiv.org/pdf/1703.06868.pdf) Arbitrary Style Transfer in Real-time with Adaptive Instance Normalization (Huang & Belongie, 2017)



**BatchNorm -> Instance Norm**



Inserts an AdaIN block after the vgg-based encoder, and uses a combined loss of **content loss** as definde above and a **style loss**. The latter compares (`MSE`) the mean and standard deviation of the encoder features of style and content on several encoding stages (after layers 4, 11, 18, and 30 of VGG).
AdaIN block forces the encoded content features to comply to encoded style features in mean and standard deviation.

The network still learns pretty fast, giving some results after *10 minutes* and relatively good results after *an hour*.  

The problems encountered include:
* **heavy artefacts** in the form of local repeated patterns in shape and color
* **cheating with color statistics**, meaning the overall color statistics are forced to compliance using strips of color on the edges of the image (exploiting the padding-born artifacts)
* **halo** effect around edges (possibly an amplified compression artifact)
* **only batch evaluation** is possible if VGG BatchNorms are not replaced with InstanceNorm

The resulting images can be found on [Google Drive](https://drive.google.com/drive/u/1/folders/1vxa1HO_UPM6Ux5MCpk_IYepY-kCG2KpY)

The implementation can be found on [GitHub](https://github.com/flying-bear/Style-Transfer/blob/main/AdaIN.ipynb).

<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/AdaIN/AdaIN_table_artifacts.jpeg?raw=true">

## Resulting Images

<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/AdaIN/bird.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/AdaIN/gate.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/AdaIN/train.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/AdaIN/sea.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/AdaIN/salad.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/AdaIN/face.png?raw=true">

# ArtGAN (simplified)


<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_scheme.png?raw=true">

[ArtGAN](https://arxiv.org/pdf/1807.10201.pdf) A Style-Aware Content Loss for Real-time HD Style Transfer (Sanakoyeu et al. 2018)

Based on the previous Autoencoder (E - D), partially implement [ArtGAN](https://compvis.github.io/adaptive-style-transfer/),
with a simple discriminator (similar to an encoder, with an additional classifyer layer compressing all chanels to one value - the similarity to style). The **ArtGAN loss** consists of three components: 
* **Discriminator Loss** $$L(E, D, Disc) = \underset{y\sim p_Y(y)}{\mathbb{E}}[log(Disc(y))] + \underset{x\sim p_X(x)}{\mathbb{E}}[log(1-Disc(D(E(x))))] + \underset{x\sim p_X(x)}{\mathbb{E}}[log(1-Disc(x))]$$ 
where $y$ is the style and $x$ - the content, and both content and stylized content are to be distinguished by discriminator from the true style images;
* **Style-Aware Content Loss** `MSE[E(x) - E(D(E(X))]`;
* **Transformed image Loss** `MSE[T(x) - T(D(E(X)))]`, where `T` is a $7 \times7$ 2d average pooling (the authors suggest trainable `T` but their own code does not implement it).

The network learns slowly (requires *up to a day* to achive meaningful results). The generated images have heavy artifacts in the form of repeated local patterns and heavily altered colors in a uniform manner across the dataset. The results were very unstable both within single image (colorwise) and across differnet images (style consistency).

Adding **discriminator balancing**, which is a threshold (0.8 from the paper) on shadowed discriminator accurcay that must be exceeded to switch to traing the generator, improved the preservance of content, but did not aliviate the local artifacts tat arised after long training (~100 epochs).

The simplified version implementation can be found on [GitHub](https://github.com/flying-bear/Style-Transfer/blob/main/ArtGAN_simlified.ipynb).
The implementation features the stage with simplified Discriminator architecture, VGG-based encoder with reflective padding, and symmetric upsampling-based Decoder, using discriminator balancing.

The resulting images can be found on [Google Drive](https://drive.google.com/drive/u/1/folders/1gnZ7J3L0a7PVSkP8jdUKiFt2rSpeOq7q)


## Resulting Images

<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/bird.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/gate.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/train.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/sea.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/salad.png?raw=true">


## Additional Examles of Artifacts at Different Stages of Implementation


<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/artifact%20bird/scales.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/artifact%20bird/horisontal%20lines%20cyan.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/artifact%20bird/horisontal%20lines%20purple.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/artifact%20bird/worms%20black.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/artifact%20bird/worms%20dark.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/artifact%20bird/worms%20red.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/artifact%20bird/worms%20warm.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/artifact%20bird/clolorscheme.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/artifact%20bird/grid%20warm.png?raw=true">
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_simplified/artifact%20bird/grid%20pale.png?raw=true">



# ArtGAN
Having proved the possibility of training simplified ArtGAN online, yet dissatisfied with the results, we turned to the full implementation of the architechture from the paper.

Importantly, the original paper training image resolution is $768\times 768$ and we use $224\times 224$ to speed up the experimentation. 

Across the network, all the convolutional layers use reflection paading followed by InstaceNorm with activation function LeakyReLU(0.2).

The Encoder is no longer pretrained. The new Ecnoder consists of four downscale blocks. Each downscale block reduces resolution to be twice lower in both dimensions, simultaneously doubling the number of channels.

The Decoder consists of two parts. The first part is six resudal blocks. Each block consits of two convolutional layers. The original article includes 9 blocks, but the number was reduced to speed up learning and accomodate for the smaller image resolution, and, thus, smaller complexity requred to reach total coverage in the receptive field. The second part of the Decoder mirrors the Encoder, thus using four upscale blocks.

The Discriminator consists of five downscale blocks. A separate classifier is used after each downscale layer, so that the Discriminator outputs five scores for the similiarity to the style on each of five depths. This helps to remove the heavy artifacts observed im the simplified model.


In the paper the Encoder and the Decoder use ReLU activation function. Replicating that resulted in mode collapse (a uniformly colored image in our case) after a few hours of training. Switching to LeakyReLU resolved this problem.

The implementation can be found on [GitHub](https://github.com/flying-bear/Style-Transfer/blob/main/ArtGAN_best.ipynb).

The resulting images can be found on [Google Drive](https://drive.google.com/drive/u/1/folders/1N8pd441V0Bth84hA-9hTgALml5xdXTaH)

Each experiment was run on Google Collab for several hours (up to 47 for the best model) with at least 12G GPU available (varying depending on collab load).

## Resulting Images

<img src = "https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/bird.png?raw=true">
<img src = "https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/gate.png?raw=true">
<img src = "https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/train.png?raw=true">
<img src = "https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/sea.png?raw=true">
<img src = "https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/salad.png?raw=true">
<img src = "https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/face.png?raw=true">


## Learing Process Illustration (form wandb dreams)
from valiant-music-4

10 epochs
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/ArtGAN_learning/0.jpg?raw=true">
200 epochs
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/ArtGAN_learning/200.jpg?raw=true">
380 epochs
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/ArtGAN_learning/380.jpg?raw=true">
400 epochs
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/ArtGAN_learning/400.jpg?raw=true">
480 epochs
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/ArtGAN_learning/480.jpg?raw=true">
1000 epochs
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/ArtGAN_learning/1000.jpg?raw=true">
5000 epochs
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/ArtGAN_learning/5000.jpg?raw=true">
10000 epochs
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/ArtGAN_learning/10000.jpg?raw=true">
25000 epochs
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/ArtGAN_learning/25000.jpg?raw=true">
43500 epochs
<img src="https://github.com/flying-bear/Style-Transfer/blob/main/example_images/ArtGAN_best/ArtGAN_learning/43500.jpg?raw=true">

# Problems




## Evaluation Protocols
The style transfer task is **inherently ill-posed**, as there is **no single correct output**.

[Jing et al. (2019)](https://arxiv.org/pdf/1705.04058.pdf) in their review of neural style transfer methods list several important evaluation criteria, including internal (loss functions) and external ones (training and application requirements). 

- Internal
  - Style & Content Loss (problematic for GANs - reset after each switch)
    - Per-Pixel Contetnt Loss functions
      - do not capture perceptual differences between output and ground-truth (e.g. identical images offset from each other by one pixel)
    - Perceptual Loss 
      - Multi-level Style & Content Loss (used in ArtGAN)
        - features extracted on several representation layers 
      - VGG-based content features vs Approximated feed-forward content features (see [Johnson et al., 2016](https://arxiv.org/pdf/1603.08155.pdf))
        - loss network remains fixed
      - Gram matrix & feature covariance matrix as style representation
        - Calibrated Style & Content Loss (see [Yeh et al., 2020](https://openaccess.thecvf.com/content_WACV_2020/papers/Yeh_Improving_Style_Transfer_with_Calibrated_Metrics_WACV_2020_paper.pdf)) 
  - Deception Rate (used in ArtGAN) 
    - VGG-based Wikiart artist classifyer deception rate
  - Expert Art Historian Evaluation (used in ArtGAN)
- External
  - Training Time
  - Inference Time
  - Computational Resource Requiremnets
  - Scalability


## Batch vs Instance Normalization
[Huang & Belongie (2017)](https://arxiv.org/pdf/1703.06868.pdf), citing [Ulyanov et al. (2017)](https://arxiv.org/pdf/1701.02096.pdf), state that replacing batch normalization (BN) with instance normalization (IN) significantly improves performance of style transfer models. The ground for that is the fact that BN forces the image statistics towards the average in each batch, which might move the statistics away from the average of the style that is to be applied to the image. Additionally, since batches might not exist during inference, so the mean and variance are pre-computed from the training set and used during inference. Therefore, changes in the target data distribution lead to issues while performing transfer learning, since the model assumes the statistics of the original training set [(Ortiz et al. 2020)](https://arxiv.org/abs/1912.05845). In practice, BN sometimes results in aggravated color artifacts. Instance Normalization, on the other hand, does not sift the statistics with respect to the single image, only with respect to parts of the image. 



## Local to Global Statistic Normalization
<img src = "https://github.com/flying-bear/Style-Transfer/blob/main/example_images/Local_to_global.png?raw=true">

The network is trained on images 224^2, so than we apply it to 2k images the following problems is arise.
Overall colors (and other features) become more boring. It is clearly seen if we compare the result with the image constructed from corresponding crops. On the third image we apply model separetly to each crop of size 224^2.

The reason for this is Instance Normalization (`nn.InstanceNorm2d`). Let us consider what is happening with fixed crop 224^2 when we apply the model to the large image. In each convolutional block (conv + norm + relu) the crop is not genuinly normalized, but rather "addaptivly normalized". I.e. the mechanism is exactly as in `AdaIN` layer: we transfer a "style" (as it called in StyleGAN papers) of global image to local crop. 
- On one hand it helps to make style transfer consistent through all the large image.
- On the other hand we don't train the model to work in such a mode. During training all the convolution blocks recieved genuinly normalized inputs. 

# Further Research Ideas & Experiments


## Local Context Normalization
[LCN](https://arxiv.org/abs/1912.05845) Local Context Normalization: Revisiting Local Normalization Ortiz et al. (2020)

Ortiz et al. (2020) proposes an interesting idea of normalizing each value in a channel according to the values in its feature group and spatial neighborhood. The paper shows the method to result in improvements across visual NN applications (such as object detection and semantic segmentation), but it would also be interesting to see how this local normalization would intearct with the semi-local nature of the style transfer task. Unfortunately, the only [implementation](https://github.com/anthonymlortiz/lcn)  is quite poor, and, according to issue list, is not maintained.

## Noise Layers

Several ideas can be borrowed from a recent famous series of papers by NVIDIA:

**[StyleGAN I.](https://arxiv.org/pdf/1812.04948.pdf)** A Style-Based Generator Architecture for Generative Adversarial Networks  (Karras et al. 2019)

- Uses `AdaIN` blocks that already are implemented above
- Adds noise on each layer in `Generator` to fight repeated patterns

**[StyleGAN II.](https://arxiv.org/pdf/1912.04958.pdf)** Analyzing and Improving the Image Quality of StyleGAN

**[StyleGAN III.](https://arxiv.org/pdf/2106.12423.pdf)** Alias-Free Generative Adversarial Networks

- Makes each block E(2) equivariant
- Works with fourier transforms of images

[Karras et al. (2019)](https://arxiv.org/pdf/1812.04948.pdf) in the first StyleGAN paper propose adding a noise layer to the Generator to combat the problem of repeating local patterns. The idea behind the proposal is that a lot of details of an image (eg. leaf or hair placement) can be randomized without affecting the overall perceptual impression if they follow the correct distribution. The authors claim that some part of the computational resources of the Generator is used to invent a way to generate spatially-varying pseudorandom numbers from earlier activations whenever they are needed. Additionally, the interdependence (universlity across space) of this pseudorandomness results in repetitive patterns in generated images. Adding per-pixel noise after each convolution attempts to both provide random information independent of previous layer activation and to spread the randomness spatially combating repeated patterns.

Unfortunately, however, in our case, adding noise layers into the every Generator layer did not visually alter the performance.


## Attention Experiment

Additional experiment attempted using 2 layers of multi-headed attention (8 heads) with LeakyReLU activation in the Residual Block part of the ArtGAN architecture instead of the convolutional layers, but that did not visually alter the performance.


# References


## Used

1.   [Agustsson & Timofte (2017)](https://people.ee.ethz.ch/~timofter/publications/Agustsson-CVPRW-2017.pdf) -- Agustsson, E., & Timofte, R. (2017). Ntire 2017 challenge on single image super-resolution: Dataset and study. In Proceedings of the IEEE conference on computer vision and pattern recognition workshops (pp. 126-135). 
2.   [Huang & Belongie (2017)](https://arxiv.org/pdf/1703.06868.pdf) -- Huang, X., & Belongie, S. (2017). Arbitrary style transfer in real-time with adaptive instance normalization. In Proceedings of the IEEE international conference on computer vision (pp. 1501-1510).
3.   [Sanakoyeu et al. (2018)](https://arxiv.org/pdf/1807.10201.pdf) -- Sanakoyeu, A., Kotovenko, D., Lang, S., & Ommer, B. (2018). A style-aware content loss for real-time hd style transfer. In proceedings of the European conference on computer vision (ECCV) (pp. 698-714). ([Additional info](https://compvis.github.io/adaptive-style-transfer/))

4. [Jing et al. (2019)](https://arxiv.org/pdf/1705.04058.pdf) -- Jing, Y., Yang, Y., Feng, Z., Ye, J., Yu, Y., & Song, M. (2019). Neural style transfer: A review. IEEE transactions on visualization and computer graphics, 26(11), 3365-3385.

5. [Johnson et al. (2016)](https://arxiv.org/pdf/1603.08155.pdf) -- Johnson, J., Alahi, A., & Fei-Fei, L. (2016, October). Perceptual losses for real-time style transfer and super-resolution. In European conference on computer vision (pp. 694-711). Springer, Cham.

6. [Yeh et al. (2020)](https://openaccess.thecvf.com/content_WACV_2020/papers/Yeh_Improving_Style_Transfer_with_Calibrated_Metrics_WACV_2020_paper.pdf) -- Yeh, M. C., Tang, S., Bhattad, A., Zou, C., & Forsyth, D. (2020). Improving style transfer with calibrated metrics. In Proceedings of the IEEE/CVF Winter Conference on Applications of Computer Vision (pp. 3160-3168).

7. [(Ortiz et al. 2020)](https://arxiv.org/abs/1912.05845) -- Ortiz, A., Robinson, C., Morris, D., Fuentes, O., Kiekintveld, C., Hassan, M. M., & Jojic, N. (2020). Local context normalization: Revisiting local normalization. In Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (pp. 11276-11285).

8. [Karras et al. (2019)](https://arxiv.org/pdf/1812.04948.pdf) -- Karras, T., Laine, S., & Aila, T. (2019). A style-based generator architecture for generative adversarial networks. In Proceedings of the IEEE/CVF conference on computer vision and pattern recognition (pp. 4401-4410).





## Further Reading


- [Gatys et al (2015)](https://arxiv.org/pdf/1508.06576.pdf) -- Gatys, L. A., Ecker, A. S., & Bethge, M. (2015). A neural algorithm of artistic style. arXiv preprint arXiv:1508.06576.


- [Li et al. (2017)](https://arxiv.org/pdf/1705.08086.pdf) -- Li, Y., Fang, C., Yang, J., Wang, Z., Lu, X., & Yang, M. H. (2017). Universal style transfer via feature transforms. Advances in neural information processing systems, 30.


- [Ulyanov et al. (2017)](https://arxiv.org/pdf/1701.02096.pdf) -- Ulyanov, D., Vedaldi, A., & Lempitsky, V. (2017). Improved texture networks: Maximizing quality and diversity in feed-forward stylization and texture synthesis. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 6924-6932).

- [Chen et al. (2017)](https://arxiv.org/pdf/1703.09210.pdf) -- Chen, D., Yuan, L., Liao, J., Yu, N., & Hua, G. (2017). Stylebank: An explicit representation for neural image style transfer. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 1897-1906). ([code](https://github.com/jxcodetw/Stylebank))

- [Yeh et al. (2018)](https://arxiv.org/pdf/1804.00118.pdf) -- Yeh, M. C., Tang, S., Bhattad, A., & Forsyth, D. A. (2018). Quantitative evaluation of style transfer. arXiv preprint arXiv:1804.00118.

- [Zhang et al. (2018)](https://arxiv.org/pdf/1801.03924.pdf) -- 
Zhang, R., Isola, P., Efros, A. A., Shechtman, E., & Wang, O. (2018). The unreasonable effectiveness of deep features as a perceptual metric. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 586-595).

- [Viazovetskyi et al. (2020)](https://www.ecva.net/papers/eccv_2020/papers_ECCV/papers/123670171.pdf) -- Viazovetskyi, Y., Ivashkin, V., & Kashin, E. (2020, August). StyleGAN2 distillation for feed-forward image manipulation. In European Conference on Computer Vision (pp. 170-186). Springer, Cham. ([Additional info](https://research.yandex.com/projects/stylegan2-distillation))

- [Voynov & Babenko (2020)](https://arxiv.org/pdf/2002.03754.pdf) -- Voynov, A., & Babenko, A. (2020, November). Unsupervised discovery of interpretable directions in the gan latent space. In International conference on machine learning (pp. 9786-9796). PMLR. ([Additional info](https://research.yandex.com/projects/latent-manipulations))


- [Karras et al. (2020)](https://arxiv.org/pdf/1912.04958.pdf) -- Karras, T., Laine, S., Aittala, M., Hellsten, J., Lehtinen, J., & Aila, T. (2020). Analyzing and improving the image quality of stylegan. In Proceedings of the IEEE/CVF conference on computer vision and pattern recognition (pp. 8110-8119).

- [Karras et al. (2021)](https://arxiv.org/pdf/2106.12423.pdf) -- Karras, T., Aittala, M., Laine, S., Härkönen, E., Hellsten, J., Lehtinen, J., & Aila, T. (2021). Alias-free generative adversarial networks. Advances in Neural Information Processing Systems, 34.


- [Hong et al. (2021)](https://arxiv.org/pdf/2108.04441.pdf) -- Hong, K., Jeon, S., Yang, H., Fu, J., & Byun, H. (2021). Domain-Aware Universal Style Transfer. In Proceedings of the IEEE/CVF International Conference on Computer Vision (pp. 14609-14617).


- [Liu et al. (2021)](https://arxiv.org/pdf/2108.03647.pdf) -- Liu, S., Lin, T., He, D., Li, F., Wang, M., Li, X., & Ding, E. (2021). Adaattn: Revisit attention mechanism in arbitrary neural style transfer. In Proceedings of the IEEE/CVF International Conference on Computer Vision (pp. 6649-6658).

- [Luo et al. (2022)](https://arxiv.org/pdf/2201.02233.pdf) -- Luo, X., Han, Z., Yang, L., & Zhang, L. (2022). Consistent Style Transfer. arXiv preprint arXiv:2201.02233. ([Demo](https://github.com/computer-vision2022/PAMA))